<a href="https://colab.research.google.com/github/MaheshGouru/CS5265-Assignment-1/blob/main/CS5265_Assignment_1_Gouru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background
# Project Description
# Performance Metric

## Clarification on Datasets needed for the Project Assignments
Just to recap some important highlights of the datasets needed for the upcoming project assignments:

- Size: at least 5000 datapoints (samples/rows/records), less than 100,000 datapoints are recommended

- #&nbsp;of columns: at least 7 attributes/features/variables (columns) are needed 

- Types of columns: a good dataset will contain not only numerical columns but also categorical columns (e.g., gender, dates, or anything else that has a finite number of categories) 

Some resources for acquiring data: 

- https://www.kaggle.com/datasets

- https://datasetsearch.research.google.com/ 

- https://archive.ics.uci.edu/ml/datasets.php 